# Projet equation d'état de disques durs en 2D par dynamique Brownienne. 

Compléter le code de BD ci-dessous en rajoutant...
- le tracé de la boite 
- les déplacements aléatoires (mais ça sort de la boîte et ça se chevauche)
- une fonction pour les rebonds (avec explications)
- une fonction pour le deoverlap (avec explications)

Utiliser le code pour une gamme de fractions volumiques et calculer des EOS 

Comparer à des modèle sur un graphique. 

Si il y a le temps : étudier l'effet de la convergence statistique, du pas de temps, de la taille finie de la boîte... 


In [165]:
from pylab import *
from time import sleep
from numpy.random import uniform
close('all') 
%matplotlib tk
ion()

L = .708e-7
d = .708e-7
dxmax = 0.1
a = 5e-9
N = 50 
eps = 1 
kT = 4.11e-21
eta = 0.001 
mobility = 1/(6*pi*eta*a)
D = mobility*kT 
V = 4*L*d
phi = pi*a**2*N/(4*L*d)
rho = N/(4*L*d)

print(phi)

dt = 0.125e-8
x = uniform(-d+a,d-a,N) 
y = uniform(-L+a,L-a,N)
Nt = 2000

def deoverlap(x, y, P):
        for i in range(N):
        #for ii in [-1,0,1]:
        #    for jj in [-1, 0, 1]:
                ii = 0
                jj = 0
                for j in range(i):
                    r = array([x[j]+ii*2*d-x[i],y[j]+jj*2*L-y[i]])
                    R = norm(r) 
                    if R<2*a:
                        shift = (2*a-R)/2
                        x[i] += -r[0]/R*shift
                        y[i] += -r[1]/R*shift
                        x[j] +=  r[0]/R*shift
                        y[j] +=  r[1]/R*shift
                        F = shift/dt / mobility * r/R
                        P += dot(r,F)
        return x, y, P
P=0
x, y, P = deoverlap(x,y,P) 

def Force(r):
    R = norm(r) 
    F = -eps*(+6/R**7 - 12/R**13) * r/R 
    return F

fig1 = figure() 
ax1 = gca() 
box = Rectangle((-d,-L),2*d,2*L,fill=False)
C = []
for i in range(N):
    C.append(Circle((x[i],y[i]),a))
    ax1.add_patch(C[-1])
ax1.add_patch(box)


P = zeros(Nt) 
#h, = plot(x,y, 'o')
axis('equal') 

figure()
ax2 = gca() 
Pline, = ax2.plot(P)

for k in range(Nt):
    dx = normal(0,1,N) 
    dy = normal(0,1,N)
    x += sqrt(2*D*dt)*dx
    y += sqrt(2*D*dt)*dy
    ## y periodic 
    #y[y>L] -= 2*L 
    #y[y<-L] += 2*L
    #x[x> d] -= 2*d 
    #x[x<-d] += 2*d
    
        # rebound
    P[k] = 0.
    for i in range(N):
        if x[i]> d-a:
            shift = 2*( x[i]-(d-a))
            x[i] -= shift
            P[k] += shift**2/dt/mobility
        if x[i]<-d+a: 
            shift = 2*((-d+a)-x[i])
            x[i] += shift 
            P[k] += shift**2/dt/mobility
        if y[i]> L-a: 
            shift = 2*( y[i]-(L-a))
            y[i] -= shift
            P[k] += shift**2/dt/mobility
        if y[i]<-L+a:
            shift = 2*((-L+a)-y[i])
            y[i] += shift
            P[k] += shift**2/dt/mobility
            
    x, y, P[k] = deoverlap(x, y, P[k]) 
    
    for i in range(N):
        if x[i]> d-a:
            shift = 2*( x[i]-(d-a))
            x[i] -= shift
            P[k] += shift**2/dt/mobility
        if x[i]<-d+a: 
            shift = 2*((-d+a)-x[i])
            x[i] += shift 
            P[k] += shift**2/dt/mobility
        if y[i]> L-a: 
            shift = 2*( y[i]-(L-a))
            y[i] -= shift
            P[k] += shift**2/dt/mobility
        if y[i]<-L+a:
            shift = 2*((-L+a)-y[i])
            y[i] += shift
            P[k] += shift**2/dt/mobility

    P[k] = P[k]/(2*V)
    ## y periodic 
    #y[y>L] -= 2*L 
    #y[y<-L] += 2*L
    #x[x> d] -= 2*d 
    #x[x<-d] += 2*d    

        # rebound
    for i in range(N):
        if x[i]> d-a: x[i] -= 2*( x[i]-(d-a))
        if x[i]<-d+a: x[i] += 2*((-d+a)-x[i])
        if y[i]> L-a: y[i] -= 2*( y[i]-(L-a))
        if y[i]<-L+a: y[i] += 2*((-L+a)-y[i])
    
    ##h.set_data(x,y)
    if (k+1)%100==0:
        #for i in range(N):
        #    C[i].center = x[i], y[i]
        Pline.set_ydata(P)
        if k>50 : 
            ax2.set_ylim(0,max(P[50:]))
        ax2.set_xlim(0,k*1.2)
        draw() 
        pause(0.1)
          
    """
    # Interactions 
    F = zeros((N,2))
    for i in range(N):
        for j in range(i):
            r = array([x[j]-x[i],y[j]-y[i]])
            F[i,:] += +Force(r)
            F[j,:] += -Force(r)
    x += dt*D/kT*F[:,0]
    y += dt*D/kT*F[:,1]
    """
    

#figure() 
#plot(P)
print(rho*kT/(1-phi)**2-rho*kT, rho*kT*(1+phi**2/8)/(1-phi)**2-rho*kT,mean(P[800:]),std(P[800:]))

0.19585442087339414
5.6004241482470665e-06 5.676420498049068e-06 5.477915939514974e-06 4.72724303802255e-06


In [174]:
def BD(phi):
    dxmax = 0.1
    a = 5e-9
    N = 50 
    eps = 1 
    kT = 4.11e-21
    eta = 0.001 
    mobility = 1/(6*pi*eta*a)
    D = mobility*kT 
    L = sqrt(N*pi*a**2/4/phi)
    d = L
    V = 4*L*d
    rho = N/(4*L*d)

    print(phi)

    dt = 0.125e-8
    x = uniform(-d+a,d-a,N) 
    y = uniform(-L+a,L-a,N)
    Nt = 2000

    def deoverlap(x, y, P):
            for i in range(N):
            #for ii in [-1,0,1]:
            #    for jj in [-1, 0, 1]:
                    ii = 0
                    jj = 0
                    for j in range(i):
                        r = array([x[j]+ii*2*d-x[i],y[j]+jj*2*L-y[i]])
                        R = norm(r) 
                        if R<2*a:
                            shift = (2*a-R)/2
                            x[i] += -r[0]/R*shift
                            y[i] += -r[1]/R*shift
                            x[j] +=  r[0]/R*shift
                            y[j] +=  r[1]/R*shift
                            F = shift/dt / mobility * r/R
                            P += dot(r,F)
            return x, y, P
    P=0
    x, y, P = deoverlap(x,y,P) 

    def Force(r):
        R = norm(r) 
        F = -eps*(+6/R**7 - 12/R**13) * r/R 
        return F

    fig1 = figure() 
    ax1 = gca() 
    box = Rectangle((-d,-L),2*d,2*L,fill=False)
    C = []
    for i in range(N):
        C.append(Circle((x[i],y[i]),a))
        ax1.add_patch(C[-1])
    ax1.add_patch(box)


    P = zeros(Nt) 
    #h, = plot(x,y, 'o')
    axis('equal') 

    figure()
    ax2 = gca() 
    Pline, = ax2.plot(P)

    for k in range(Nt):
        dx = normal(0,1,N) 
        dy = normal(0,1,N)
        x += sqrt(2*D*dt)*dx
        y += sqrt(2*D*dt)*dy
        ## y periodic 
        #y[y>L] -= 2*L 
        #y[y<-L] += 2*L
        #x[x> d] -= 2*d 
        #x[x<-d] += 2*d

            # rebound
        P[k] = 0.
        for i in range(N):
            if x[i]> d-a:
                shift = 2*( x[i]-(d-a))
                x[i] -= shift
                P[k] += shift**2/dt/mobility
            if x[i]<-d+a: 
                shift = 2*((-d+a)-x[i])
                x[i] += shift 
                P[k] += shift**2/dt/mobility
            if y[i]> L-a: 
                shift = 2*( y[i]-(L-a))
                y[i] -= shift
                P[k] += shift**2/dt/mobility
            if y[i]<-L+a:
                shift = 2*((-L+a)-y[i])
                y[i] += shift
                P[k] += shift**2/dt/mobility

        x, y, P[k] = deoverlap(x, y, P[k]) 

        for i in range(N):
            if x[i]> d-a:
                shift = 2*( x[i]-(d-a))
                x[i] -= shift
                P[k] += shift**2/dt/mobility
            if x[i]<-d+a: 
                shift = 2*((-d+a)-x[i])
                x[i] += shift 
                P[k] += shift**2/dt/mobility
            if y[i]> L-a: 
                shift = 2*( y[i]-(L-a))
                y[i] -= shift
                P[k] += shift**2/dt/mobility
            if y[i]<-L+a:
                shift = 2*((-L+a)-y[i])
                y[i] += shift
                P[k] += shift**2/dt/mobility

        P[k] = P[k]/(2*V)
        ## y periodic 
        #y[y>L] -= 2*L 
        #y[y<-L] += 2*L
        #x[x> d] -= 2*d 
        #x[x<-d] += 2*d    

            # rebound
        for i in range(N):
            if x[i]> d-a: x[i] -= 2*( x[i]-(d-a))
            if x[i]<-d+a: x[i] += 2*((-d+a)-x[i])
            if y[i]> L-a: y[i] -= 2*( y[i]-(L-a))
            if y[i]<-L+a: y[i] += 2*((-L+a)-y[i])

        ##h.set_data(x,y)
        if (k+1)%100==0:
            #for i in range(N):
            #    C[i].center = x[i], y[i]
            Pline.set_ydata(P)
            if k>50 : 
                ax2.set_ylim(0,max(P[50:]))
            ax2.set_xlim(0,k*1.2)
            draw() 
            pause(0.1)
    
    Pstd = std(P[800:])
    Pout = mean(P[800:])
    return Pout, Pstd

phi = 0.2 
P, Pstd = BD(phi)
print(rho*kT/(1-phi)**2-rho*kT,P) 

0.2


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/_backend_tk.py", line 252, in idle_draw
    self.draw()
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super().draw()
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py", line 436, in draw
    self.figure.draw(self.renderer)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/artist.py", line 74, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/artist.py", line 51, in draw_wrapper
    return draw(artist, renderer)
  File "/opt/anaconda3/lib/python3.7/site-packages/

KeyboardInterrupt: 

In [179]:
#phi = linspace(0.05, 0.3, 4)
#Pnum = []
#Pstdnum = [] 
#for phil in phi:
#    P, Pstd = BD(phil)
#    Pnum.append(P) 
#    Pstdnum.append(Pstd)
    
Vp = pi*a**2
    
Pnum = array(Pnum)
Pstdnum = array(Pstdnum)
Pid   = phi/Vp*kT 
P = Pid+Pnum
Pp = P + Pstdnum 
Pm = P - Pstdnum 

figure() 
plot(phi, P, 'ob')
plot(phi, Pp, '^b')
plot(phi, Pm, 'vb')
phith=linspace(0.01,0.4,100); 
rhoth = phith/(pi*a**2)
plot(phith, rhoth*kT, '--k')
plot(phith, rhoth*kT/(1-phith)**2)
plot(phith, rhoth*kT*(1+phith**2/8)/(1-phith)**2)

In [176]:
close('all') 